### Importing

In [1]:
from utils import dataGetter
from datetime import datetime
import pandas as pd
import numpy as np


### Getting data, data will be stored in /tmp/history.zip on a Unix base file system

In [26]:
data = dataGetter.getStockData(datetime(2017,11,14))
#data = dataGetter.getDataFromFile("/path/to/history.zip")

### Getting floating data

In [88]:
float_data = pd.read_csv( 'data/vn30_floating.csv',index_col=0,dtype={'totalShares':np.int64,'floating':np.int64})

vn30_members = float_data.index.values
vn30_data = data[(data.Ticker.isin(vn30_members))]

float_data['non_float'] = float_data['totalShares'] - float_data['floating']
float_data['f_ratio_0'] = (float_data['floating']/ float_data['totalShares']*20.0).round()/20.0
float_data['f_ratio_1'] = (((float_data['floating'] - float_data['non_float']) / float_data['floating'])*20.0).round()/20.0
float_data

,totalShares,floating,non_float,f_ratio_0,f_ratio_1
﻿ticker,,,,,
CTD,77050000,76534844,515156,1.00,1.00
BMP,81860938,81860938,0,1.00,1.00
VCB,3597768575,3597768575,0,1.00,1.00
FPT,530961105,530878729,82376,1.00,1.00
STB,1885215716,1803653429,81562287,0.95,0.95
DHG,130746071,130746071,0,1.00,1.00
KBC,475711167,469760189,5950978,1.00,1.00
GAS,1913950000,1913348070,601930,1.00,1.00
ROS,472999999,472999999,0,1.00,1.00


In [104]:
#create a copy, not a slice of old data
vn30_data = vn30_data.copy()
vn30_data['FloatingShares'] = vn30_data['Ticker'].map(float_data['floating'])
vn30_data['TotalShares'] = vn30_data['Ticker'].map(float_data['totalShares'])
vn30_data['MktCap'] = vn30_data['Close']*vn30_data['TotalShares']
vn30_data['f_0'] = vn30_data['Ticker'].map(float_data['f_ratio_0'])
vn30_data['f_1'] = vn30_data['Ticker'].map(float_data['f_ratio_1'])


### Calculating $c_i$

In [105]:
#use the business day before the third monday of July
calcDate = datetime(2017,7,14)
calcData = vn30_data[vn30_data['Date'] == calcDate]

stop = False
Z = 0.1
c = pd.Series(1, index=calcData.index)

while not stop:
    totalMktCap = (calcData['MktCap']*c).sum()
    mktCap = calcData['MktCap']*c
    mktCap_pct = mktCap / totalMktCap
    tickers = calcData['Ticker'].values
    weightMap = dict(zip(tickers, mktCap_pct.values))
    mktCapMap = dict(zip(tickers, mktCap.values))

    J = [x for x in weightMap if weightMap[x] <= Z ]
    I = calcData[calcData['Ticker'].isin(J) ]['MktCap'].sum()/totalMktCap    
    nJ = [x for x in weightMap if weightMap[x]>Z]
    if( len( nJ ) == 0 ):
        stop = True
        break
    psf = calcData['Close']*calcData['f_0']*calcData['TotalShares']
    psf_J = psf[calcData['Ticker'].isin(J)].sum()
    new_c = Z/I*psf_J/psf[calcData['Ticker'].isin(nJ)]
    #print new_c
    c[new_c.index] = new_c
#c

In [106]:
tickers = calcData['Ticker']
c_map = dict( zip( tickers.values, c.values ))

In [107]:
cmv = vn30_data['Close'] * vn30_data['TotalShares'] * vn30_data['f_0']* vn30_data['Ticker'].map(c_map)
vn30_data['CMV'] = cmv

In [108]:
#vn30_data['vn30_comp'] = vn30_data['Close'] * vn30_data['IndexWeight']
vn30_data.groupby(['Date'])['CMV'].sum()/1.707455e+09

Date
2000-07-28      0.329007
2000-07-31      0.335163
2000-08-02      0.341337
2000-08-04      0.347511
2000-08-07      0.353667
2000-08-09      0.359841
2000-08-11      0.366015
2000-08-14      0.372189
2000-08-16      0.378345
2000-08-18      0.384519
2000-08-21      0.390694
2000-08-23      0.396849
2000-08-25      0.390694
2000-08-28      0.384519
2000-08-30      0.378345
2000-09-01      0.372189
2000-09-06      0.366015
2000-09-08      0.359841
2000-09-11      0.366015
2000-09-13      0.372189
2000-09-15      0.378345
2000-09-18      0.372189
2000-09-20      0.378345
2000-09-22      0.372189
2000-09-25      0.372189
2000-09-27      0.372189
2000-09-29      0.374241
2000-10-02      0.374241
2000-10-04      0.374241
2000-10-06      0.374241
                 ...    
2017-10-04    769.726875
2017-10-05    769.865914
2017-10-06    771.948402
2017-10-09    772.649639
2017-10-10    774.350037
2017-10-11    778.645794
2017-10-12    781.644509
2017-10-13    789.149385
2017-10-16    786.01